In [6]:
from ultralytics import YOLO

model = YOLO('yolov5s.pt')  


PRO TIP  Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



In [7]:

 # run the model 
#results = model("https://img.parlmany.com/large/32219.jpg")

#results = model("https://paultan.org/image/2020/01/UKM-plate-Malaysia-1-630x420.jpg") # this is high quality image 
#results = model("https://5.imimg.com/data5/SELLER/Default/2023/8/337432643/BW/SR/KG/78628874/seat-belt-detection-system-250x250.jpg")
results = model("https://source.roboflow.com/R8EEE0Tjxmf6Qxxmeqa3eqWlknt2/02xSLnHetDx47FWi3jQA/original.jpg")
print(results[0])
predictions = results[0]



Found https://source.roboflow.com/R8EEE0Tjxmf6Qxxmeqa3eqWlknt2/02xSLnHetDx47FWi3jQA/original.jpg locally at original.jpg
image 1/1 c:\Users\w\Documents\my_csv_project\ai-pros-v1-2025\W4_DL\C1_M3_Data_Management\C1M3_Assignment\original.jpg: 384x640 1 car, 311.5ms
Speed: 37.9ms preprocess, 311.5ms inference, 44.1ms postprocess per image at shape (1, 3, 384, 640)
ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseba

In [8]:
import os
from PIL import Image
# get the cars only and save it 
boxes = results[0].boxes

 #output_dir = 'detected_cars'
 #os.makedirs(output_dir, exist_ok=True)
cars =[]

car_count = 0
for box in boxes:
    
    cls = int(box.cls[0])
    
    if cls == 2:
        car_count += 1
        
        cord = box.xyxy[0].tolist()
        #
        img = Image.fromarray(results[0].orig_img)
        cropped_car = img.crop((cord[0], cord[1], cord[2], cord[3]))
        cars.append(cropped_car)
        
        # cropped_car.save(os.path.join(output_dir, f'car_{car_count}.jpg'))

print(f"   {len(cars) }  saved  at   ")
print(cars)

   1  saved  at   
[<PIL.Image.Image image mode=RGB size=664x664 at 0x24AC5F07B00>]


In [9]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
# get the plates and presetnt the image 

model = YOLO(r"C:\Users\w\Documents\my_csv_project\ai-pros-v1-2025\W4_DL\C1_M3_Data_Management\C1M3_Assignment\license-plate-finetune-v1m.pt")

license_plates_crops = []

for x in cars:
    results = model.predict(source=x)
    
 
    res_plotted = results[0].plot()
    
    plt.figure(figsize=(10, 6))
    plt.imshow(res_plotted)
    plt.axis('off') 
    plt.show() 
    
    
    for r in results:
        for box in r.boxes:
            cord = box.xyxy[0].tolist()
            plate_crop = x.crop((cord[0], cord[1], cord[2], cord[3]))
            license_plates_crops.append(plate_crop)

print(f"  {len(license_plates_crops)} plates croped  .")


0: 640x640 1 License_Plate, 931.6ms
Speed: 6.4ms preprocess, 931.6ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)


<Figure size 1000x600 with 1 Axes>

  1 plates croped  .


In [10]:
import pytesseract
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO


pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
tess_config = r'--tessdata-dir "C:\Program Files\Tesseract-OCR\tessdata" --psm 7'


plate_model = YOLO(r"C:\Users\w\Documents\my_csv_project\ai-pros-v1-2025\W4_DL\C1_M3_Data_Management\C1M3_Assignment\license-plate-finetune-v1m.pt")
seatbelt_model = YOLO(r"C:\Users\w\Desktop\best.pt")


for i, car_img in enumerate(cars):
    print(f"\n--- {i+1} ---")
    
   
    belt_results = seatbelt_model.predict(source=car_img, conf=0.25)
    has_violation = False
    
    for r in belt_results:
        for box in r.boxes:
            cls_id = int(box.cls[0])
           
            if cls_id in [9]: 
                has_violation = True
                print(f" parson no seatbelt! (Confidence: {box.conf[0]:.2f})")
    
 
    if has_violation:
        plate_results = plate_model.predict(source=car_img)
        
        for pr in plate_results:
            for p_box in pr.boxes:
               
                cord = p_box.xyxy[0].tolist()
                plate_crop = car_img.crop((cord[0], cord[1], cord[2], cord[3]))
                
                
                plate_np = np.array(plate_crop)
                plate_np = cv2.resize(plate_np, None, fx=3, fy=3, interpolation=cv2.INTER_CUBIC)
                gray_plate = cv2.cvtColor(plate_np, cv2.COLOR_RGB2GRAY)
                
              
                text = pytesseract.image_to_string(gray_plate, lang='ara', config=tess_config)
                print(f" plate number   : {text.strip()}")
                
              
                plt.imshow(pr.plot())
                plt.title(f"Violation Detected - Plate: {text.strip()}")
                plt.show()
    else:
        print("No belt or violation was detected in this image.")


--- 1 ---

0: 640x640 (no detections), 149.8ms
Speed: 4.4ms preprocess, 149.8ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)
No belt or violation was detected in this image.
